In [30]:
import requests 
import json 
import pprint

In [31]:
class Report: 
    def __init__(self):
        
        self.url ="https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest" #link per i report
        self.params = {
            'start': '1', #inizia dalla prima cripto in classifica
            'limit': '100',  #finisce con la 100'esima cripto in classifca
            'convert': "USD"  #valuta delle cripto -> dolalto 
        }
        
        self.headers = {
            'Accepts': 'application/json', #formato della risposta 
            'X-CMC_PRO_API_KEY': 'e5c4253e-a74e-4274-84a0-9ed17d09b04f' #chiave per poter scaricare i dati 
        }
        
    def fetchCurrentData(self): 
        r = requests.get(url = self.url, headers = self.headers, params = self.params).json() #fa la richieste per i dati 
        return r['data'] #restituisce i dati in formato json 


In [45]:
report = Report()

# currencies = report.fetchCurrentData()
# with open("output.json", "w") as outfile: 
#     json.dump(currencies, outfile, indent = 4)

data = json.load(open("output.json"))


In [93]:
def max_volume(data): #trova la valuta con maggiore volume delle ultime 24h
    volumes = {v['quote']['USD']['volume_24h']: v['name'] for v in data} #dizionario con chiave volume e valore nome
    
    return volumes[max(volumes)] #ritorna nome della cripto con volume maggiore


Tether


In [126]:
def best_and_worst_cripto(data): #trova le migliori e le peggiori 10 criptovalute (per incremento in percentuale nelle ultime 24 ore)

    volumes = { v['quote']['USD']['percent_change_24h']: v['name'] for v in data} #dizionario con chiave volume e valore nome
    
    sorted_volumes = sorted(volumes) #lista ordinata in base alle chiavi del dizionario
    best_volumes = sorted_volumes[:10]  #lista con i volumi migliori
    worst_volumes = sorted_volumes[-10:] #lista con i volumi peggiori
    
    best = []
    worst = []
    
    for v,name in volumes.items():  #ciclo sul dizionario 
        if v in best_volumes:  #se la chiave è nella lista dei migiori 
            best.append(name)  #salvo il nome della cripto 
        if v in worst_volumes: 
            worst.append(name)
            
    return best, worst


['Algorand', 'Aave', 'Ren', 'Verge', 'Komodo', 'SwissBorg', 'Bancor', 'Hive', 'MCO', 'WAX']
['UNUS SED LEO', 'Compound', '0x', 'Synthetix Network Token', 'Nexo', 'ZB Token', 'Quant', 'Loopring', 'Celsius', 'BitShares']


In [ ]:
def 